# BMI565: Bioinformatics Programming & Scripting

#### (C) Michael Mooney (mooneymi@ohsu.edu)

## Week 11: Benchmarking and Optimizing Python Code

1. Benchmarking / Profiling in Python
2. Optimizing Code with Cython
3. Parallel Processing
    - `Multiprocessing` module
    - `pp` (Parallel Python) module
4. Final Exam Review

#### Requirements

- Python 2.7
- `time`, `timeit`, and `profile` modules
- `scipy` and `numpy` modules
- `multiprocessing` module
- Parallel Python, `pp`, module

** Note: Parallel Python can be installed with `pip install pp`.

In [1]:
import numpy as np

## Benchmarking / Profiling

There are a number of ways to evaluate the performance of your Python code. Three useful modules are:

- `time`
- `timeit`
- `profile`

In [2]:
## Define a function that determines if a number is prime
def isprime(n):
    """
    Returns the number if it is prime, otherwise returns None.
    """
    assert n > 0, "Number must be greater than 0!"
    if n < 2: return None
    for i in range(2,n):
        if n % i == 0:
            return None
    return n

def get_primes(min, max):
    result = []
    possible_primes = range(min,max+1)
    for n in possible_primes:
        result.append(isprime(n))

    prime_nums = [n for n in result if n is not None]
    return prime_nums

In [3]:
## Binary search function
def bsearch(l, n):
    s = 0
    e = len(l) - 1
    while True:
        if s > e:
            return None
        mid = (s + e)/2
        if l[mid] < n:
            s = mid  + 1
        elif l[mid] > n:
            e = mid  - 1
        else:
            return mid

In [4]:
## Recursive binary search function
def rec_bsearch(l,n,s=0,e=None):
    if e is None: e = len(l) - 1
    if s > e:
        return None
    mid = (s + e)/2
    if n == l[mid]:
        return mid
    elif n < l[mid]:
        return rec_bsearch(l,n,s,mid-1)
    else:
        return rec_bsearch(l,n,mid+1,e)

### `time` module

In [5]:
import time

def search_time(fun, N, M):
    runtimes = []
    nums = np.arange(M)
    start_time = time.time()
    for i in range(N):
        t0 = time.time()
        cmd = fun + "(nums, 3450)"
        idx = eval(cmd)
        runtimes.append(time.time() - t0)
    
    print "Total runtime: ", time.time() - start_time
    print "Mean runtime: ", sum(runtimes)/len(runtimes)
    return None

In [6]:
print "Binary Search:"
search_time("bsearch", 5000, 1000000)

Binary Search:
Total runtime:  0.119760990143
Mean runtime:  2.35820770264e-05


### `timeit` module

In [7]:
import timeit

## Get the runtime of a Python statement
timeit.timeit("bsearch(nums, 3450)", setup="from __main__ import bsearch; import numpy as np; nums = np.arange(1000000)", number=5000)

0.06543111801147461

In [8]:
## Create a timer and run it multiple times
timer = timeit.Timer("bsearch(nums, 3450)", setup="from __main__ import bsearch; import numpy as np; nums = np.arange(1000000)")
timer.repeat(3, number=5000)

[0.06094789505004883, 0.056292057037353516, 0.05849313735961914]

### `profile` module

In [9]:
import profile
nums = np.arange(10000000)
profile.run("rec_bsearch(nums, 3450)")

         23 function calls (5 primitive calls) in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 :0(len)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
     19/1    0.000    0.000    0.000    0.000 <ipython-input-4-3e47e1fd46da>:2(rec_bsearch)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        0    0.000             0.000          profile:0(profiler)
        1    0.000    0.000    0.001    0.001 profile:0(rec_bsearch(nums, 3450))




## Cython

Cython is a programming language that allows you to create C extensions for your Python programs, thereby improving performance. I'll show one simple example of how you might do this below, but for more details consult the Cython documentation (link in references). Before running the example code blocks below, you'll need to create two files and run a command to compile the Cython module. Note that the code shown below for the Cython implementation of the binary search algorithm is nearly identical to pure Python, with just a few minor tweaks (mostly accounting for data type conversions between Python/Numpy and Cython). 

Create a file called **'cy_bsearch.pyx'** that contains the following:

    #cython: boundscheck=False
    #cython: wraparound=False
    cimport numpy as np
    
    cdef bsearch(np.ndarray[np.int64_t, ndim=1] L, int target, int start=0, end=None):
        """
        Binary search
        """
        cdef Py_ssize_t idx, mid
        
        if end is None: 
            end = len(L)-1
        while L.size > 0:
            idx = start + (end - start)/2
            mid = (len(L)-1)/2
            if L[mid] == target:
                return idx
            else:
                if target < L[mid]:
                    L = L[:mid]
                    end = idx - 1
                else:
                    L = L[mid+1:]
                    start = idx + 1
        return None


Create another file called **'setup.py'** that contains the following:

    from distutils.core import setup
    from distutils.extension import Extension
    from Cython.Distutils import build_ext
    
    import numpy
    
    ext = Extension("cy_bsearch", ["cy_bsearch.pyx"],
                    include_dirs = [numpy.get_include()])
    
    setup(ext_modules=[ext],
          cmdclass = {'build_ext': build_ext})

Finally, run the following at the command-line (this may produce a number of warnings, which can be ignored):

    python setup.py build_ext --inplace

This example is purely meant to illustrate how you might use Cython. However, because binary search is such a simple algorithm, we don't see any improvement in performance. This is an important point when it comes to optimization: **Over-optimization can be a real waste of time.** Rather than optimizing every last piece of code, you should focus only on those places where you will see real benefit.

In [10]:
import pyximport
pyximport.install(setup_args={'include_dirs':[np.get_include()]})
from cy_bsearch import bsearch as cython_bsearch

In [11]:
print "Cython Binary Search:"
search_time("cython_bsearch", 5000, 1000000)

Cython Binary Search:
Total runtime:  0.179076910019
Mean runtime:  3.5307598114e-05


In [12]:
timeit.timeit("cython_bsearch(nums, 3450)", setup="from cy_bsearch import bsearch as cython_bsearch; import numpy as np; nums = np.arange(1000000)", number=5000)

0.11948013305664062

## Parallel Processing

Parallel processing is a technique for improving the performance of a computational task, based on the idea that large problems can often be split into multiple smaller problems. These smaller problems can then be solved simultaneously (in parallel). Given the constraints of processor design and development, parallel computing (multi-processor machines) is now a common way to improve computational power.

There are numerous Python modules that allow you to take advantage of the computational power of multiple processors (the list below is not complete):

[https://wiki.python.org/moin/ParallelProcessing](https://wiki.python.org/moin/ParallelProcessing)

Keep in mind that there is always some amount of overhead cost due to splitting a large task into multiple smaller task and then gathering/compiling the individual results. The efficiency gains achieved with parallel processing will depend on the individual tasks being performed and the amount of communication (data transfer) required.

In [13]:
import multiprocessing as mp
import pp

In [14]:
## Find prime numbers serially
min_prime = 30000
max_prime = 50000

t0 = time.time()
prime_nums = get_primes(min_prime, max_prime)
t1 = time.time()

print "There are %d prime numbers between %d and %d." % (len(prime_nums), min_prime, max_prime)
print "Elapsed time:", t1 - t0

There are 1888 prime numbers between 30000 and 50000.
Elapsed time: 11.9381160736


In [15]:
profile.run("get_primes(min_prime, max_prime)")

         60008 function calls in 12.435 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    20001    0.041    0.000    0.041    0.000 :0(append)
    20002    4.154    0.000    4.154    0.000 :0(range)
        1    0.000    0.000    0.000    0.000 :0(setprofile)
        1    0.095    0.095   12.435   12.435 <ipython-input-2-4d94308a6ebf>:13(get_primes)
    20001    8.146    0.000   12.300    0.001 <ipython-input-2-4d94308a6ebf>:2(isprime)
        1    0.000    0.000   12.435   12.435 <string>:1(<module>)
        1    0.000    0.000   12.435   12.435 profile:0(get_primes(min_prime, max_prime))
        0    0.000             0.000          profile:0(profiler)




### `Multiprocessing` module

`Multiprocessing` is a module in Python's standard library that allows you to spawn multiple Python processes. It is an easy way to take advantage of multiple cores on a single machine.

[https://docs.python.org/2/library/multiprocessing.html](https://docs.python.org/2/library/multiprocessing.html)

In [16]:
## Get number of CPUs
mp.cpu_count()

8

In [17]:
## Find prime numbers using parallel processes
possible_primes = range(min_prime,max_prime+1)

t2 = time.time()
pool = mp.Pool(processes=4)
result2 = pool.map(isprime, possible_primes)
prime_nums2 = [n for n in result2 if n is not None]
t3 = time.time()

## Make sure to close the processes created by Pool
pool.close()

print "There are %d prime numbers between %d and %d." % (len(prime_nums2), min_prime, max_prime) 
print "Elapsed time:", t3 - t2

There are 1888 prime numbers between 30000 and 50000.
Elapsed time: 3.64277696609


### `pp` (Parallel Python) module

The Parallel Python module can be used to parallelize across multiple processors on a single machine, and also across multiple nodes of a computing cluster.

[http://www.parallelpython.com/](http://www.parallelpython.com/)

In [18]:
## Create pp job server
job_server = pp.Server(ncpus=7)
jobs = []

t4 = time.time()
## Submit jobs to pp server
for i in possible_primes:
    jobs.append(job_server.submit(isprime, (i,)))
## Wait for all jobs to finish
job_server.wait()
prime_nums3 = [job() for job in jobs if job() is not None]
t5 = time.time()

## Close the processes created by pp
job_server.destroy()

## Print results
print "There are %d prime numbers between %d and %d." % (len(prime_nums3), min_prime, max_prime) 
print "Elapsed time:", t5 - t4

There are 1888 prime numbers between 30000 and 50000.
Elapsed time: 6.73694086075


## References

- [https://wiki.python.org/moin/ParallelProcessing](https://wiki.python.org/moin/ParallelProcessing)
- [http://docs.cython.org/en/latest/](http://docs.cython.org/en/latest/)
- [https://docs.python.org/2/library/time.html](https://docs.python.org/2/library/time.html)
- [https://docs.python.org/2/library/timeit.html](https://docs.python.org/2/library/timeit.html)
- [https://docs.python.org/2/library/profile.html](https://docs.python.org/2/library/profile.html)

#### Last Updated: 12-Sep-2017